In [66]:
import numpy as np
import pandas as pd
from fourinarowfunctions import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Provide the directory where the data is kept

In [67]:
direc = 'C:/Users/svo/Documents/FourinarowData/Data/fourinarow-development/'

Read parameters and calculate feature drop rate and heuristic quality

In [68]:
df = pd.read_csv(direc + 'paramsMatrix.csv')
params = np.array([expand_params(r) for r in df.values[:,-10:].astype(float)])

df['feature drop rate'] = params[:,-1]
df['heuristic quality'] = np.apply_along_axis(get_heuristic_quality,1,params)

We compute planning depth on the cluster. First, export the parameters

In [69]:
np.savetxt(direc + 'params.txt',params)

Create a directory on the cluster and move params.txt into it. Edit the compute_planning_depth.sh shell script so that the directory is correct, and change the number of jobs to the number of lines in params.txt (in this case 370). Run the script, then zip the depth folder, move it back to your local computer and unzip in the directory where paramsMatrix.csv is stored. 

In [78]:
filename = direc + 'depth.npy'
if os.path.exists(filename):
    depth = np.load(filename,allow_pickle=True)
else: 
    depth = np.vstack([np.loadtxt(direc + 'depth/depth_fourinarow-dev_'+str(i//5)+'_' + str(i%5+1) + '.txt')
             for i,_ in enumerate(params)])
    np.save(filename,depth,allow_pickle=True)

In [80]:
df['depth'] = depth

In [81]:
df

,Unnamed: 0,subject,fold,1,2,3,4,5,6,7,8,9,10,feature drop rate,heuristic quality,depth
0,0,1,1,9.99910,0.001066,0.436090,0.050846,0.41127,0.93515,1.70390,2.313400,6.6399,9.9652,0.436090,0.638619,8.13583
1,1,1,2,1.33510,0.026620,0.273650,0.050997,0.59204,0.91980,0.79910,0.752940,3.5729,9.9878,0.273650,0.559960,2.92211
2,2,1,3,7.48720,0.838520,0.055475,0.053560,0.39363,0.73419,1.17780,0.880200,3.9903,9.9874,0.055475,0.596631,0.04544
3,3,1,4,7.02990,0.001067,0.162850,0.050095,0.25001,0.72710,1.70470,0.939260,3.8773,9.9999,0.162850,0.525318,9.13304
4,4,1,5,10.00000,1.000000,0.312920,0.050222,0.33533,0.85563,1.64490,1.131400,5.9752,9.1561,0.312920,0.616708,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,365,74,1,1.10000,0.001002,0.141220,0.050000,0.93175,0.89058,0.62378,0.022125,3.2012,9.9966,0.141220,0.494260,6.92550
366,366,74,2,0.98406,0.001000,0.227380,0.050000,0.94070,0.87151,0.82328,0.206620,4.6212,10.0000,0.227380,0.540146,8.48107
367,367,74,3,0.70346,0.003957,0.278730,0.050000,1.20140,1.01880,0.85624,0.364240,4.2530,9.9633,0.278730,0.490209,4.94710
368,368,74,4,1.73180,0.083501,0.252810,0.050015,1.24200,1.06840,0.73644,-0.191930,4.3865,9.9975,0.252810,0.466085,1.54808


Calculate Elo

In [71]:
results_df = pd.read_csv(direc + 'forElo.csv')
results = [[row['subject'].split(':')[1],str(row['level']),row['outcome']] if row['user_color']=='black' else
           [str(row['level']),row['subject'].split(':')[1],-row['outcome']] for _,row in results_df.iterrows()]
create_bayeselo_input(results,direc + 'games.pgn')

In [72]:
bayeselo_direc = 'C:/Users/svo/Documents/Sourcetree repos/fourinarow/Analysis notebooks/new/'
tournament_direc = 'C:/Users/svo/Documents/Sourcetree repos/fourinarow/'

elo = run_bayeselo(bayeselo_direc,[tournament_direc + 'tournament_results_short.pgn', direc + 'games.pgn'])
elo = {key : val for key,val in elo.items() if key.startswith('R_')}

In [73]:
elo

{'R_3NEgCFlK1BFSmdZ': 332,
 'R_3KxYOzYEKloODvs': 295,
 'R_3EEYoLyNN9SpuuT': 287,
 'R_1DSHShmupLFd4m9': 285,
 'R_zUdkqlxVsz3sXcJ': 250,
 'R_31SM3mwQvAhBwar': 240,
 'R_D1Afhly6eOCY66B': 235,
 'R_3fDgQlApbcv1n4e': 232,
 'R_0OFsszTuxG152et': 220,
 'R_1Do3QQWN7pWkPwQ': 217,
 'R_2uPQbPniskmS0Ep': 213,
 'R_NVGtaI93MLLVQfT': 208,
 'R_OOl4OrOpRgbrLBD': 205,
 'R_2ahKzS67s2u094R': 192,
 'R_vdABJ1XvXNriwff': 192,
 'R_2QRpawEy8w6txvN': 184,
 'R_28G5rUmynDONqbJ': 178,
 'R_2X10rcLujzFpHxv': 176,
 'R_2zuodaGDxpO8dfe': 164,
 'R_24rdiskqYQc7BsO': 155,
 'R_1LeQOpZ9pat1k7D': 153,
 'R_10BfVTQovUcZioM': 151,
 'R_21j62EoiJ63yEJ5': 150,
 'R_2q8z9DfvMUHDZio': 142,
 'R_3j2tErwVrBPXWAH': 140,
 'R_Q0Jd5lCBrQu5Dm9': 139,
 'R_12lQaNUgCpqQizh': 139,
 'R_3lJsCIPqfpqRhkf': 138,
 'R_3NR4vGxEweQ74vU': 132,
 'R_25zHKt5eqk2Fr1k': 131,
 'R_3kzZPXnjFGbpt30': 129,
 'R_2XpQzVOPaMJjkCo': 123,
 'R_d1i4xMt2Vzivxg5': 120,
 'R_2wvlbfkxX6e4M29': 115,
 'R_2qlLx30lmNEXUJL': 115,
 'R_eINRkRhqhKHRjzz': 109,
 'R_1esokoelkIOTy89': 109,
 